In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/notebook32e0aa5c2c/__results__.html
/kaggle/input/notebook32e0aa5c2c/__notebook__.ipynb
/kaggle/input/notebook32e0aa5c2c/__output__.json
/kaggle/input/notebook32e0aa5c2c/custom.css
/kaggle/input/notebook32e0aa5c2c/dataset_processed/documents_processed.csv


# Load the preprocessed data

In [2]:
import os
import numpy as np
import pandas as pd

In [3]:
documents = pd.read_csv('/kaggle/input/notebook32e0aa5c2c/dataset_processed/documents_processed.csv')
documents.head ()

,document_id,sentences
0,0,መሰላችሁ አንባቢያን በተደጋጋሚ ጥሪው ደርሷት ልትታደመው ያልቻለችው የአለ...
1,5,አይመስለኝም ሃኒ ከውድድሩ ውጭ የሆነችው የቢግ ብራዘርስ አብሮ የመኖር ው...
2,10,ይባላል የፈረንጅ ስምና የፈረንጅ መድሀኒት ያላቸው በሽታዎችም ይይዙን ጀመ...
3,15,ወይስ ኋኧረ ይስጡይ ባለ ንብረት እየነጠቀው ኋባይሆን ለጠንቋይ እሸጠዋለሁ...
4,20,አላቸው እና እሳቸው ኋእሱ እኛን የሆነው ወይስ ነን እሱን የሆንነው ወይስ...


# Convert the data to TaggedDocuments for Doc2Vec

In [4]:
from gensim.models.doc2vec import TaggedDocument

In [5]:
tagged_documents = []
for index, row in documents.iterrows ():
    tagged_document = TaggedDocument (words = row['sentences'].split (" "), tags = [row['document_id']])
    tagged_documents.append (tagged_document)

In [6]:
len(tagged_documents)

699085

In [7]:
tagged_documents[123]

TaggedDocument(words=['አሰበ', 'ሊፈርድ', 'ነው', 'መለሰለት', 'ያ', 'ድምጽ', 'በማን', 'አሰበ', 'ባንተ', 'መለሰ', 'ድምፁ', 'ትንሹ', 'ቆይቶ', 'ሰአቱ', 'ደርሶአል', 'አለው', 'ፈልጐ', 'ሳይፈልግ', 'ሜዳው', 'አመራ', 'ሜዳው', 'መሃል', 'ከአጃቢዎቹ', 'ቆመ', 'የሁሉም', 'አይን', 'የተተከለው', 'ነበር', 'ያናግርሃል', 'ያ', 'ድምጽ', 'ደቂቃዎች', 'አለፉ', 'ህዝቡ', 'እየተንጫጫ', 'ነበር', 'ድንገት', 'ሰማዩ', 'በአንድ', 'ታይቶ', 'በማይታወቅ', 'እጅግ', 'ልዩ', 'የሆነ', 'ብርሃን', 'ተደበላለቀ', 'ሁሉም', 'ፀጥ', 'እረጭ', 'አለ', 'በፍርሃት', 'ተርበድብዶ', 'ሳይወድ', 'በግዱ', 'መሬቱ', 'በግንባሩ', 'ተደፋ', 'አይኑ', 'ማየት', 'አዕምሮውም', 'ማሰብ', 'አልቻለም', 'ያየው', 'ብርሃን', 'በሱ', 'የማሰብ', 'አቅም', 'የሚታሰብ', 'አልነበረም', 'ያየኸው', 'ፈጣሪን', 'የከበበውን', 'ብርሃን', 'ነው', 'ፈጣሪን', 'አላየውም', 'አለው', 'ያ', 'ድምጽ', 'በግንባሩ', 'እንደተደፋ', 'ነው', 'ለዚህ', 'ላየው', 'ግርማ', 'ብርሃን', 'ለዝንተ', 'አለም', 'በግንባሩ', 'ቢደፋ', 'እንደማይጠላ', 'በድንገት', 'ታሰበው', 'ምንነቱን', 'መግለጽ', 'የማይቻል', 'እጅግ', 'ውብ', 'ግርማ', 'ያለውና', 'እፎይታ', 'የሚሰጥ', 'ድምጽ', 'ሰማ', 'ቃላቶቹ', 'አይሰሙም', 'እንደፈለገ', 'ይገባል', 'ጉዳይህን', 'እናያለን', 'የሚል', 'ይመስላል', 'ሰው', 'ይሁን', 'መልአክት', 'የማይታወቅ', 'ጮክ', 'ሰማው', 'ጌታ', 'ሆይ', 'ሁሉን', 'ቻይና', 'ሁሉን', 'አዋቂ', 'ነህ', 'ሰው', 'አንተን', 'ደፍሮአል',

In [8]:
import gc

del documents
gc.collect()

0

# Train the model 

In [9]:
from gensim.models.doc2vec import Doc2Vec

In [10]:
model = Doc2Vec (vector_size = 100, window = 5, min_count = 5, workers = 8, epochs = 20, alpha = 0.03, min_alpha = 0.002)

In [11]:
model.build_vocab(tagged_documents)

In [12]:
model.corpus_count

699085

In [13]:
model.train(tagged_documents, total_examples = model.corpus_count, epochs = model.epochs)

# Save the model

In [14]:
import os

model_dir = "models"
model_path = os.path.join(model_dir, "doc2vec_am.bin")

os.makedirs(model_dir, exist_ok=True)

model.save(model_path)
